##Funciones de apoyo
Ejecute estas celdas para preparar algunas características del ambiente que serán necesarias para la práctica

In [ ]:
!pip install mne

In [ ]:
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
subjects = [1]  # may vary
runs = [3]  # may vary
raw_fnames = eegbci.load_data(subjects, runs)
raws = [read_raw_edf(f, preload=True) for f in raw_fnames]
# concatenate runs from subject
raw = concatenate_raws(raws)
# make channel names follow standard conventions
eegbci.standardize(raw)


datos=[]
eventos=[[],[],[]]
for i in range(64):
    datos.append((raw[0][1][0:], raw[i][0][0][0:]))
for i in range(len(raw.annotations)):
  if raw.annotations[i]['description'] == 'T0':
    eventos[0].append(raw.annotations[i]['onset'])
  if raw.annotations[i]['description'] == 'T1':
    eventos[1].append(raw.annotations[i]['onset'])
  if raw.annotations[i]['description'] == 'T2':
    eventos[2].append(raw.annotations[i]['onset'])



## Sección 2. Registro de eventos conductuales
De manera similar a los datos fisiológicos, los datos conductuales pueden tener miles de maneras de registrarse y contemplarse. No obstante, a menudo podemos simplificar su registro limitándonos a presencia o ausencia.

En el conjunto de datos que estamos utilizando, se registró la actividad cerebral de sujetos mientras realizaban diversas actividades motoras.

Durante esta sección, como en la anterior, trabajaremos con datos eléctricos capturados por 64 electrodos. Esta vez será importante considerar el acomodo de los mismos, que puede verificar en la liga siguiente: https://physionet.org/content/eegmmidb/1.0.0/64_channel_sharbrough.pdf

Añadiremos también la información de que al sujeto se le pide abrir y cerrar la mano correspondiente con una señal en la pantalla de una computadora. La celda siguiente grafica esta información:

- Las líneas verticales en amarillo representan el inicio de periodos de reposo,
- las líneas en azul representan el inicio de periodos de actividad con la mano izquierda,
- y las líneas en rojo representan el inicio de periodos de actividad con la mano derecha.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

electrodo = 61

k = electrodo - 1
datamin= np.min(datos[k][1])
datamax= np.max(datos[k][1])
plt.figure(figsize= (100, 10))
plt.axes(xlabel= "Tiempo (s) @ 160Hz" , ylabel= "Potencial (V)")
plt.vlines(eventos[0], ymin=datamin, ymax=datamax, colors = 'y')
plt.vlines(eventos[1], ymin=datamin, ymax=datamax, colors = 'b')
plt.vlines(eventos[2], ymin=datamin, ymax=datamax, colors = 'r')

plt.plot(datos[k][0],datos[k][1], color='0')

#### Ejercicio 2.1

---
Revise la distribución de electrodos, explore el comportamiento de las señales provenientes de diferentes electrodos (modificando la línea donde se indica el electrodo, de la celda anterior) y conteste lo que se pide.

2.1.1 Responda: ¿De qué grupo de electrodos esperaríamos observar incrementos de voltaje durante un periodo de actividad con la mano derecha? ¿Por qué? (Hay varias respuestas válidas)

In [ ]:
#

2.1.2 Responda: Al comparar las señales del electrodo 61 y el 22, ¿qué diferencias encuentra?


In [ ]:
#

### Ejemplo 2.1: Ruido, bucles y condicionales

Probablemente se dio cuenta de que el electrodo 22 y el electrodo 24 tienen algunos picos muy curiosos, muy regulares. Estos picos son, para nuestros propósitos, ruido, ya que no provienen de la actividad cerebral sino _muscular_. Se trata de parpadeos.

Esta es una buena oportunidad para practicar la automatización de tareas repetitivas.

Si queremos realizar la misma operación muchas veces en lugares ligeramente distintos, resulta útil crear bucles (loops). Los bucles son bloques de código que se repiten una y otra vez hasta cambiar alguna situación determinada por el programador.

Los bucles "_for_" se repiten mientras avanzan en alguna secuencia de números u objetos en un contenedor, como en el caso siguiente:

In [ ]:
# Podemos escribir un bucle que imprima cada elemento de una lista, uno por uno,
# a la vez que realiza una suma no relacionada:

listaEj = ["ola", "io", "soi", "1", "lista", "pero", "nadien", "m", "kiere", "):"]
sumita = 1
vuelta = 0
for elem in listaEj:
  print(elem)
  sumita = sumita + 4
  vuelta = vuelta + 1
  print("la sumita lleva", sumita)
  print("este bucle se repitió ", vuelta, " veces")


Note que la sintaxis sigue esta estructura:

for x in y:

       código

Debe ponerse " : " al final de la linea que llama el bucle, y el código debe escribirse con una sangría que lo separa de las líneas originales.



Los bucles "_while_" aunque usan una sintaxis similar, se repiten mientras que una _condición_ asignada sea verdadera.

Debe tener MUCHO cuidado si decide utilizar bucles _while_, ya que es enteramente posible producir un bucle que se repite para siempre, hasta que usted elija forzar la detención de la celda. (O la computadora explote)

In [ ]:

# Si el bloque de código no es capaz de cambiar la condición
# que mantiene su bucle, nunca se detendrá.

sumita = 0
while sumita < 10:
  print(sumita)
  sumita = sumita + 1

Note que los bucles _while_ necesitan poner atención al resultado de una __operación lógica__. En la celda anterior, estamos pidiéndole a Python que, mientras sea verdad que la variable "sumita" tiene un valor menor a 10, puede volver a ejecutar el bucle.

Una vez que "sumita" tiene un valor de 10, sería falso decir que es menor que 10, así que el bucle termina.

A la operación lógica que determina si un bloque de código debe ejecutarse le llamamos una __condición.__ Podemos poner condiciones utilizando también la palabra clave _if._



In [ ]:

# Esta condición siempre se cumple
if 1 < 2 :
  print("1 es menor que 2")

# Esta condición nunca se cumple
if 1 > 2 :
  print("1 es mayor que 2")

Combinando conocimiento de bucles y condiciones, podemos, por ejemplo, navegar nuevamente la lista que usamos al comenzar este ejemplo, para después imprimir un mensaje de respuesta cuando encontramos una cadena en particular:

In [ ]:
for elem in listaEj:
  print("Estoy inspeccionando el elemento:", elem, "Y estoy buscando a:", "m")
  if elem == "m":
    print("AQUÍ ESTÁS")
    break
  else:
    print("Tú no eres")



Note que en cada "vuelta" (cada iteración) del bucle, la variable "elem" cambia su valor, reflejando el de cada elemento de listaEj.

Además, note que es posible concluir un bucle prematuramente utilizando la palabra clave _break_

#### Ejercicio 2.2
---

Con su flamante nuevo conocimiento de bucles, estudie el código de las funciones de apoyo y genere los bucles que se piden.


2.2.1 Escriba un bucle que, en una sola variable, sume todos los números pares desde 0 hasta 100. Haga que el código imprima "hola" y el valor de su variable cada vez que el bucle se repite.

In [ ]:
#

2.2.2 Escriba un bucle que, en una sola variable, sume todos los números enteros desde 0 a 100 EXCEPTO los múltiplos de 7. Imprima su variable al inicio de cada iteración.

In [ ]:
#

### Ejemplo 2.3: Datos dañados o faltantes
Con alarmante frecuencia, encontraremos que los datos con los que trabajamos podrían haber sido registrados con errores, pérdidas o algún otro tipo de daño.

En esos casos, lo ideal es la recolección de un registro nuevo. Sin embargo, la realidad es que esto a menudo no es feasible. De manera para el análisis tenemos esencialmente dos alternativas: __ignorar la señal dañada__, _condenándola al olvido_, o bien, __reparar la señal__.

La reparación, que denominamos __imputación__ es posible tanto para conjuntos de datos "estáticos" como para series de tiempo, aunque las técnicas utilizadas varían un poco. En cualquiera de los casos, la intención es hacer ajustes a los datos de manera que no introduzcamos un sesgo ni alteremos las propiedades generales del conjunto.

En la siguiente celda generaremos un conjunto de datos que simula el registro de las edades y estaturas de 80 personas, donde algunos datos han sido "perdidos" al azar.

In [ ]:
import numpy as np

aleaGen = np.random.default_rng()

artDatos = []
edades = aleaGen.integers(low = 15, high=88, size = 80)
tallas = aleaGen.uniform(low = 1.34, high= 2.35, size = 80)

artDatos.append(edades)
artDatos.append(tallas)

damage1 = aleaGen.integers(low=0,high=79,size=aleaGen.integers(low=1,high=20,size=1))
damage2 = aleaGen.integers(low=0,high=79,size=aleaGen.integers(low=1,high=20,size=1))

for i in damage1:
  artDatos[0][i] = 0
for i in damage2:
  artDatos[1][i] = 0

In [ ]:
#Puede revisar el contenido de los datos en esta celda

artDatos[0]

In [ ]:
plt.plot(artDatos[1])

Para este tipo de datos, a menudo es suficiente sustituir los valores faltantes por la media de los valores que sí tenemos.

El siguiente bloque de código ejemplifica cómo hacerlo:

In [ ]:
# Primero necesitamos obtener la media de las edades y las tallas, sin contar los faltantes
x = 0    #Usamos una variable temporal donde guardaremos la suma de nuestros valores
contador = 0 #Otra variable temporal para usar como contador
for i in range(len(artDatos[0])):
  if artDatos[0][i] != 0: # Esta condición revisará que el valor sea diferente de cero
    contador = contador + 1
    x = x + artDatos[0][i]
promedio_edad = x / contador
print("Se encontraron", len(artDatos[0]) - contador, "edades faltantes")

x = 0    #Reiniciamos variables temporales
contador = 0
for i in range(len(artDatos[1])):
  if artDatos[1][i] != 0:
    contador = contador + 1
    x = x + artDatos[1][i]
promedio_talla = x / contador
print("Se encontraron", len(artDatos[1]) - contador, "tallas faltantes")


print("El promedio de edades es", promedio_edad)
print("El promedio de estaturas es", promedio_talla)

In [ ]:
#Haremos una copia de los datos originales para poder compararlos con nuestro resultado
artDatos_original = []
artDatos_original.append(artDatos[0].copy())
artDatos_original.append(artDatos[1].copy())

Ahora solo nos falta reemplazar los valores 0 por el promedio que calculamos...

In [ ]:
for i in range(len(artDatos[0])):
  if artDatos[0][i] == 0:
    artDatos[0][i] = promedio_edad

for i in range(len(artDatos[1])):
  if artDatos[1][i] == 0:
    artDatos[1][i] = promedio_talla

In [ ]:
artDatos_original[0]

In [ ]:
artDatos[0]

In [ ]:
artDatos_original[1]


In [ ]:
artDatos[1]

#### Ejercicio 2.3
----
Calcule, imprima y grafique los valores promedio (incluyendo los valores faltantes esta vez) tanto para los datos originales como en los datos procesados, señale cómo son similares o diferentes.

In [ ]:
#

### Conclusiones

Es vital asegurarnos de que no existen datos faltantes agregando ruido a nuestros datos, por lo que es un paso irreemplazable en el preprocesamiento.


Para las series de tiempo, la técnica que podemos utilizar para corregir puntos de información faltantes es la __interpolación__. Consiste en tomar los puntos más cercanos en el tiempo al punto faltante, y promediar esos.

A menudo existen funciones para interpolar automáticamente. Aprenderemos su uso en prácticas posteriores.

